In [ ]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import gdal
import imageio
from PIL import Image
import json
import os
from scipy.ndimage import gaussian_filter
from skimage import data, img_as_float, measure, exposure
from skimage.morphology import reconstruction
from skimage.transform import rescale, resize
import torch
from torchvision import datasets, models, transforms
from torch.autograd import Variable
from torch import nn, optim, cuda
import cv2
from cv2 import contourArea
from shapely.geometry import Polygon, MultiPolygon, MultiPoint, Point
from shapely.ops import nearest_points
from shapely.ops import unary_union
import scipy as sp
import scipy.ndimage
import itertools
from itertools import combinations

!pip install geojson
import geojson
from geojson import FeatureCollection, Feature, dump

!pip install geopandas
import geopandas

!pip install rasterio
from rasterio import mask
import rasterio

!pip install alphashape
import alphashape

import gc
from shapely import affinity

     |████████████████████████████████| 972kB 8.7MB/s 
     |████████████████████████████████| 14.8MB 204kB/s 
     |████████████████████████████████| 6.5MB 50.7MB/s 
     |████████████████████████████████| 18.3MB 174kB/s 


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
print(device)

from google.colab import drive

drive.mount('/content/drive')

cuda
Mounted at /content/drive


In [ ]:
model_version = "alex_8"
params_file = model_version + '_params.pth' #model version to use

#get model
model_dir = '/content/drive/My Drive/Lake detection/models/'

params = torch.load(model_dir + params_file)
model_name = params['name']

preprocess = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
                                     ])

In [ ]:
def make_folder(folder): #creates a new directory if it does not exist
    directory = os.path.dirname(folder)
    if not os.path.exists(directory):
        os.makedirs(directory)

def re_orderbands(b1, b2, b3): #reprders image bands
    img_new = np.zeros((b1.shape[0], b1.shape[1],3))
    img_new[:,:,0] = b1
    img_new[:,:,1] = b2
    img_new[:,:,2] = b3
    return img_new

def normalize_band(data,dmin, dmax): #Normalize: min = 0, max = 1
    data_n = np.zeros(data.shape)
    data_n = (data - dmin)/(dmax - dmin)
    return data_n

def loadImages(file): #load S1 and S2 data, return geographic info and data
        raster = gdal.Open(file)
        data = raster.ReadAsArray()
        gray = data[0,:,:] #grayscale
        HV = data[1,:,:] #HV
        
        gray[gray > 1] = 1
        HV[HV > 1] = 1
        HV[HV < 0] = 0
        gray[gray == 0] = np.nan
        HV[np.isnan(gray)] = np.nan
        HV[(np.isnan(HV)) & (~np.isnan(gray))] = np.nanmean(HV) #gets rid of nan strips in S1 data

        datas = np.zeros((HV.shape[0],HV.shape[1],2))
        datas[:,:,0] = gray  
        datas[:,:,1] = HV    
        
        ulx, xres, xskew, uly, yskew, yres  = raster.GetGeoTransform()
        lrx = ulx + (raster.RasterXSize * xres)
        lry = uly + (raster.RasterYSize * yres)
        x_minmax = (ulx, lrx)
        y_minmax = (uly, lry)

        #image shape
        m = datas.shape[0]
        n = datas.shape[1]

        #x and y coordinates for each pixel
        xx = np.linspace(x_minmax[0], x_minmax[1], n) 
        yy = np.linspace(y_minmax[0], y_minmax[1], m)
        
        return datas, m, n, xx, yy

def get_mtns(all_data, mt):
    mtns = np.zeros(all_data[:,:,0].shape)
    for item in mt:
        i = item[0]
        j = item[1]
        dx = item[2]

        temp_img = all_data[i:i+dx, j:j+dx, 0]
        temp_mtns = np.zeros(temp_img.shape)
        temp_mtns[temp_img < 0.4] = 1

        mtns[i:i+dx, j:j+dx] = temp_mtns

    if len(mt) > 5:
        mtns[all_data[:,:,0] < 0.4] = 1
    #mtns = open_space(mtns, np.ones((5,5)), 1)
    
    mtns = cv2.dilate(mtns,np.ones((20,20)),iterations = 1)
    return mtns


def fill_holes(img, kernel,i): #morphological closing operation
    closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations = i)
    return closing

def open_space(img, kernel,i): #morphological opening operation
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations = i)
    return opening

def filter_lake(img): #perform morphological operations
    kernel1 = np.ones((3,3),np.uint8)
    kernel2 = np.ones((6,6),np.uint8)
    kernel3 = np.ones((12,12),np.uint8)

    th2 = fill_holes(img, kernel1, 1)
    th2 = open_space(th2, kernel1, 1)
    th3 = fill_holes(th2, kernel2, 1)
    th3 = open_space(th3, kernel2, 1)
    th4 = open_space(th3, kernel3, 1) 
    th4 = fill_holes(th4, kernel3, 1) 

    return th4

def thresholding(img):#create binary image based on threshold
      v2 = np.percentile(img, 5)
      new_img = np.zeros(img.shape)
      new_img[img < v2] = 1
      return new_img

def convert_poly(co, xx, yy): #convert polygon of indexes to geo-located polygon
    geo_contour = list()
    for ii in range(len(co)):
        y,x = co[ii]
        if int(y) < 1:
            new_y = yy[0]
        elif int(y) >= len(yy)-1:
            new_y = yy[-1]
        elif int(y) == y:
            new_y = yy[int(y)]
        else:
            temp1 = int(y - 0.5)
            temp2 = int(y + 0.5)
            new_y = (yy[temp1] + yy[temp2])/2

        if int(x) < 1:
            new_x = xx[0]
        elif int(x) >= len(xx)-1:
            new_x = xx[-1]
        elif int(x) == x:
            new_x = xx[int(x)]
        else:
            temp1 = int(x - 0.5)
            temp2 = int(x + 0.5)
            new_x = (xx[temp1] + xx[temp2])/2
            
        tup = new_x, new_y
        geo_contour.append(tup)
    return geo_contour
    
def buffer_poly(poly, file, b, band): #create a buffer around the polygon
    multipoly = MultiPolygon([poly])
    new_multipoly = MultiPolygon([Polygon(poly.buffer(b), [poly.exterior.coords])])
    with rasterio.open(file) as src:
        out_image1, out_transform1 = rasterio.mask.mask(src, multipoly, crop = True)
        out_image1 = np.squeeze(out_image1)
        out_image1[out_image1 == 0] = np.nan
        out_image2, out_transform2 = rasterio.mask.mask(src, new_multipoly, crop = True)
        out_image2 = np.squeeze(out_image2)
        out_image2[out_image2 == 0] = np.nan
    if len(out_image1.shape) > 2:
        return out_image1[band,:,:], out_image2[band,:,:]
    else:
        return None, None

def edge_check(ilake): #check to see if lake is too close to edge of image
    good = False
    maxy = int(np.max(ilake[:,0]))
    maxx = int(np.max(ilake[:,1]))
    miny = int(np.min(ilake[:,0]))
    minx = int(np.min(ilake[:,1]))

    r = np.min([maxy-miny, maxx-minx, 100])
    temp_img = all_data[miny-r:maxy+r, minx-r:maxx+r,1]

    if np.mean(temp_img) > -5:
        good = True  

    return good

def mtn_check(ilake, properties):
    good = True
    l = convert_poly(ilake, xx, yy)
    p = Polygon(l)
    im, b = buffer_poly(p, file, 1000,1)
    im2, b2 = buffer_poly(p, file, 1000,0)

    if len(mt) > 10 and np.nanmin(b2) < 0.4: good = False
   
    maxy = int(np.max(ilake[:,0]))
    maxx = int(np.max(ilake[:,1]))
    miny = int(np.min(ilake[:,0]))
    minx = int(np.min(ilake[:,1]))
    r = 5

    temp_img = mtns[miny-r:maxy+r, minx-r:maxx+r]

    if np.mean(temp_img) > 0:
        good = False

    ran = np.nanmax(im2) - np.nanmin(im2)
    if ran > 0.75:
        good = False

    # fig, ax = plt.subplots(1,5,figsize = (20,5))
    # ax[0].imshow(im, cmap = 'gray', vmin = 0, vmax = 1)
    # ax[1].imshow(b, cmap = 'gray', vmin = 0, vmax = 1)
    # ax[2].imshow(im2, cmap = 'gray', vmin = 0, vmax = 1)
    # ax[3].imshow(b2, cmap = 'gray', vmin = 0, vmax = 1)
    # ax[4].imshow(temp_img, cmap = 'gray', vmin = 0, vmax = 1)

    return good

def get_means(im, b):

    HV_mean = np.nanmean(im)
    im_non_nans = (~np.isnan(im)).sum()
    total_non_nans = (~np.isnan(b)).sum()
    b_non_nans = total_non_nans - im_non_nans
    total_mean = np.nanmean(b)
    buffer_mean = (total_non_nans*total_mean - im_non_nans*HV_mean)/b_non_nans
    return HV_mean, buffer_mean


def list_to_json(lst, pth): #write list to json file
    _json = {'type': 'FeatureCollection', 'features':lst}
    json_string = json.dumps(_json)
    with open(pth, 'w') as json_file:
        json.dump(_json, json_file)

def refine_lakes1(contoured_lakes):
    full_im = np.zeros(all_data[:,:,1].shape)
    lakes = list()
    remaining_lakes = list()

    for ilake in contoured_lakes:
        #ilake = contoured_lakes[i]
        l = convert_poly(ilake, xx, yy)
        p = Polygon(l)
        im, b = buffer_poly(p, file, 1000,1)

        if im is not None:
            HV_mean, buffer_mean = get_means(im, b)

            maxy = int(np.max(ilake[:,0]))+15
            maxx = int(np.max(ilake[:,1]))+15
            miny = int(np.min(ilake[:,0]))-15
            minx = int(np.min(ilake[:,1]))-15

            xxtemp = xx[minx:maxx]
            yytemp = yy[miny:maxy]
            image = all_data[miny:maxy, minx:maxx,1]

            th = np.zeros(image.shape)
            th[(image - buffer_mean) < -0.2] = 1

            kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
            kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(6,6))
            th2 = fill_holes(th, kernel1,1)
            th3 = open_space(th2, kernel1,1)
            th4 = fill_holes(th3, kernel1,1)
            th5 = open_space(th4, kernel1,1)
            th5 = cv2.blur(th5,(3,3))
            th5[th5 > 0] = 1
            th6 = fill_holes(th5, kernel2,1)
            th7 = open_space(th6, kernel2,1)
            th7 = cv2.blur(th7,(6,6))
            th7[th7 > 0] = 1

            full_im[miny-2:maxy-2, minx-2:maxx-2] = th7

    full_im[full_im < 1] = 0
    full_im[full_im >= 1] = 1

     #contour lakes
    contoured_lakes = measure.find_contours(full_im, 0.5) #contour 'lakes'
    del full_im

    for co in contoured_lakes:
        properties = get_stats(co, 0)
        l = convert_poly(co, xx, yy)
        p = Polygon(l)
        p2 = affinity.scale(p, xfact = 0.9, yfact = 0.9)
        coords = list(zip(*p2.exterior.coords.xy))
        d = int(len(coords)/25)
        if d > 1:
            coords = coords[0::d]
            coords.append(coords[0])

        if check_lake(properties, 0) and mtn_check(co, properties) and edge_check(co):
            coords = [coords]
            if properties['area'] > 250000 and properties['cvh_ratio'] > 0.95 and properties['Q4'] < -0.15 and properties ['Q3'] < -0.17 and properties['background_diff'] < -0.15 and properties['w_l_ratio'] < 2.5: properties['code'] = 'round'
            elif properties['area'] > 250000 and properties['cvh_ratio'] > 0.9 and properties['Q4'] < -0.16 and properties['background_diff'] < -0.16 and properties['w_l_ratio'] < 2.5: properties['code'] = 'round'
            elif properties['area'] > 1500000 and properties['background_diff'] < -0.15 and properties['Q4'] < -0.14 and properties['Q1'] < -0.2:  properties['code'] = 'big'
            elif properties['area'] > 1000000 and properties['background_diff'] < -0.19 and properties['Q4'] < -0.19 and properties['Q1'] < -0.2:  properties['code'] = 'big'
            elif properties['area'] < 150000 and properties ['area'] > 50000 and properties['Q4'] < -0.2 and properties['cvh_ratio'] > 0.8 and properties['w_l_ratio'] < 2.5: properties['code'] = 'small'
            #elif properties['w_l_ratio'] > 2.5 and properties['background_diff'] < -0.2 and properties['Q4'] < -0.2 and properties['area'] > 150000: properties['code'] = 'long'
            elif properties['HV_mean'] < 0.3 and properties['Q4'] < -0.16 and properties['background_diff'] < -0.18 and properties['area'] > 50000 and properties['w_l_ratio'] < 2.5: properties['code'] = 'dark'
            else: properties['code']  = 'good'
            poly_dict = {'type':'Polygon', 'coordinates':coords}
            geom = {'type':'Feature', 'geometry':poly_dict, 'properties':properties}
            lakes.append(geom)
        elif properties['area'] > 50000 and properties['background_diff'] < -0.1:
            remaining_lakes.append(co)
  
    del contoured_lakes
    return lakes, remaining_lakes

def refine_lakes2(contoured_lakes):
    full_im = np.zeros(all_data[:,:,1].shape)
    lakes = list()
    remaining_lakes = list()

    for ilake in contoured_lakes:
        l = convert_poly(ilake, xx, yy)
        p = Polygon(l)
        im, b = buffer_poly(p, file, 1000,1)

        if im is not None:
            HV_mean, buffer_mean = get_means(im, b)

            maxy = int(np.max(ilake[:,0]))+15
            maxx = int(np.max(ilake[:,1]))+15
            miny = int(np.min(ilake[:,0]))-15
            minx = int(np.min(ilake[:,1]))-15

            xxtemp = xx[minx:maxx]
            yytemp = yy[miny:maxy]
            image = all_data[miny:maxy, minx:maxx,1]

            th = np.zeros(image.shape)
            th[(image - buffer_mean) < -0.2] = 1

            kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
            kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(6,6))
            kernel3 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(12,12))
            th2 = fill_holes(th, kernel1,1)
            th3 = open_space(th2, kernel1,1)
            th4 = fill_holes(th3, kernel1,1)
            th5 = open_space(th4, kernel1,1)
            th6 = fill_holes(th5, kernel2,1)
            th7 = open_space(th6, kernel2,1)
            if len(xxtemp) * len(yytemp) > 30000:
                th7 = fill_holes(th7, kernel3,1)
                th7 = open_space(th7, kernel3,1)

        full_im[miny-2:maxy-2, minx-2:maxx-2] = th7

    full_im[full_im < 1] = 0
    full_im[full_im >= 1] = 1

     #contour lakes
    contoured_lakes = measure.find_contours(full_im, 0.5) #contour 'lakes'
    del full_im

    for co in contoured_lakes:
        properties = get_stats(co,1)
        l = convert_poly(co, xx, yy)
        p = Polygon(l)
        im, b = buffer_poly(p, file, 1000, 1)

        coords = list(zip(*p.exterior.coords.xy))
        d = int(len(coords)/25)
        if d > 1:
            coords = coords[0::d]
            coords.append(coords[0])
        if check_lake(properties, 1) and mtn_check(co, properties) and edge_check(co):
            coords = [coords]
            if properties['area'] > 250000 and properties['cvh_ratio'] > 0.95 and properties['Q4'] < -0.15 and properties ['Q3'] < -0.17 and properties['background_diff'] < -0.15 and properties['w_l_ratio'] < 2.5: properties['code'] = '1 round'
            elif properties['area'] > 250000 and properties['cvh_ratio'] > 0.9 and properties['Q4'] < -0.16 and properties['background_diff'] < -0.16 and properties['w_l_ratio'] < 2.5: properties['code'] = '1 round'
            elif properties['area'] > 1500000 and properties['background_diff'] < -0.15 and properties['Q4'] < -0.14 and properties['Q1'] < -0.2:  properties['code'] = '1 big'
            elif properties['area'] > 1000000 and properties['background_diff'] < -0.19 and properties['Q4'] < -0.19 and properties['Q1'] < -0.2:  properties['code'] = '1 big'
            elif properties['area'] < 150000 and properties ['area'] > 50000 and properties['Q4'] < -0.2 and properties['cvh_ratio'] > 0.8 and properties['w_l_ratio'] < 2.5: properties['code'] = '1 small'
            else: properties['code']  = '1 good'
            poly_dict = {'type':'Polygon', 'coordinates':coords}
            geom = {'type':'Feature', 'geometry':poly_dict, 'properties':properties}
            lakes.append(geom)

    return lakes

def get_stats(ilake, code):
    properties = {}

    lake = convert_poly(ilake, xx, yy)
    poly = Polygon(lake) 
    if code == 0:
        poly = affinity.scale(poly, xfact = 0.9, yfact = 0.9)
    properties['area'] = poly.area #7) area
    convex_hull = poly.convex_hull
    cvh_area = convex_hull.area
    properties['cvh_ratio'] = poly.area/cvh_area #8) poly to convex hull poly area ratio
    
    im, b = buffer_poly(poly, file, 1000,1)
    if im is not None:
        HV_mean, buffer_mean = get_means(im, b)
        properties['HV_mean'] = HV_mean.astype(float) #1) average HV value
        properties['background_diff'] = (HV_mean - buffer_mean).astype(float) #difference of lake from background

        maxy = int(np.max(ilake[:,0]))
        maxx = int(np.max(ilake[:,1]))
        miny = int(np.min(ilake[:,0]))
        minx = int(np.min(ilake[:,1]))
        rangex = maxx-minx
        rangey = maxy-miny

        Q1 = np.nanmean(all_data[miny-rangey:maxy-rangey, minx:maxx,1])
        Q2 = np.nanmean(all_data[miny+rangey:maxy+rangey, minx:maxx,1])
        Q3 = np.nanmean(all_data[miny:maxy, minx-rangex:maxx-rangex,1])
        Q4 = np.nanmean(all_data[miny:maxy, minx+rangex:maxx+rangex,1])
        Qs = [HV_mean - Q1, HV_mean - Q2, HV_mean - Q3, HV_mean - Q4]
        Qs.sort()
        properties['Q1'] = Qs[0].astype(float) #4-7 quadrant differences
        properties['Q2'] = Qs[1].astype(float)
        properties['Q3'] = Qs[2].astype(float)
        properties['Q4'] = Qs[3].astype(float)

        box = poly.minimum_rotated_rectangle
        xbox, ybox = box.exterior.coords.xy
        edge_length = (Point(xbox[0], ybox[0]).distance(Point(xbox[1], ybox[1])), Point(xbox[1], ybox[1]).distance(Point(xbox[2], ybox[2]))) # get length of bounding box edges
        length = max(edge_length)
        width = min(edge_length)
        properties['w_l_ratio'] = length/width

    else:
        properties['HV_mean'] = 0
        properties['background_diff'] = 0
        properties['Q1'] = 0
        properties['Q2'] = 0
        properties['Q3'] = 0
        properties['Q4'] = 0
        properties['w_l_ratio'] = 0

    return properties

def check_lake(properties, t):
    round_lake = False
    if properties['area'] > 250000 and properties['cvh_ratio'] > 0.95 and properties['Q4'] < -0.1575 and properties ['Q3'] < -0.17 and properties['background_diff'] < -0.1575 and properties['w_l_ratio'] < 2.5: round_lake = True
    elif properties['area'] > 250000 and properties['cvh_ratio'] > 0.9 and properties['Q4'] < -0.168 and properties['background_diff'] < -0.168 and properties['w_l_ratio'] < 2.5: round_lake = True
    big_lake = False
    if properties['area'] > 1500000 and properties['background_diff'] < -0.1575 and properties['Q4'] < -0.147 and properties['Q1'] < -0.2:  big_lake = True
    elif properties['area'] > 1000000 and properties['background_diff'] < -0.1995 and properties['Q4'] < -0.1995 and properties['Q1'] < -0.2:  big_lake = True
    small_lake = False
    if properties['area'] < 150000 and properties ['area'] > 50000 and properties['Q4'] < -0.21 and properties['cvh_ratio'] > 0.8 and properties['w_l_ratio'] < 2.5: small_lake = True
    dark_lake = False
    if t == 0:
        #if properties['w_l_ratio'] > 2.5 and properties['background_diff'] < -0.2 and properties['Q4'] < -0.2 and properties['area'] > 150000: long_lake = True
        if properties['HV_mean'] < 0.3 and properties['Q4'] < -0.16 and properties['background_diff'] < -0.18 and properties['area'] > 50000 and properties['w_l_ratio'] < 2.5: dark_lake = True
    #cond1 = properties['area'] > 750000 or (properties['area'] < 750000 and properties['area'] > 150000 and properties['cvh_ratio'] > 0.7)
    cond2 = properties['Q4'] < -0.1785 and properties['Q3'] < -0.20 and properties['background_diff'] < -0.21 and properties ['area'] > 150000# and properties['w_l_ratio'] < 2.5
    cond3 = True
    if big_lake and properties['background_diff'] > -0.2 and (properties['Q4'] - properties['background_diff']) < -0.01: cond3 = False
    if not big_lake and properties['background_diff'] > -0.22 and (properties['Q4'] - properties['background_diff']) < -0.01: cond3 = False
    
    
    return (cond2 or round_lake or big_lake or small_lake or dark_lake) and cond3


In [ ]:
regions = ['CW', 'SW', 'NW', 'NO', 'NE', 'SE']
year = '2019'

region = 'SE'
if year == '2018' or year == '2019':
    tifs_folder = '/content/drive/My Drive/Lake detection/greenland/' + year + '/' + region[0:2] + '/original_data/' #original data location
    tiles_folder = '/content/drive/My Drive/Lake detection/greenland/' + year + '/' + region[0:2]  + '/lake_tiles/' #location to store tiles
else:
    tifs_folder = '/content/drive/My Drive/Lake detection/greenland/' + year + '/original_data/' #original data location
    tiles_folder = '/content/drive/My Drive/Lake detection/greenland/' + year + '/lake_tiles/' #location to store tiles
make_folder(tiles_folder) #make folder to store subsurface lake image tiles
dxs = [256,512]

files = sorted(glob(tifs_folder + region + '*np.tif'))
#files = [files[-1]]
lakes = list()
num = 0

for file in files:
    mt = list()
    print(file)
    print('loading model')
    model = torch.load(model_dir + model_name)
    for param in model.parameters():
        param.requires_grad = False
    model.eval();

    print("loading data")
    all_data, m, n, xx, yy = loadImages(file) #get original image, pixel dimensions (m,n) and x and y coordinates for each pixel (xx, yy)
    full_im = np.zeros((m,n))

    for dx in dxs:
        print(f'{dx}: classifying images')
        for k in range(0,m-dx,int(dx/2)):
            y1 = yy[k+dx]
            y2 = yy[k]
            for j in range(0,n-dx,int(dx/2)):
                f_name = region + '_' + str(num) + '_' + str(dx) + '.jpg'                    
                x1 = xx[j]
                x2 = xx[j+dx]
                temp_img = all_data[k:k+dx,j:j+dx,:] #tile
                temp_img = resize(temp_img, (256, 256),anti_aliasing=True) #resize

                if np.mean(temp_img) > -5:                      
                    HV_norm = normalize_band(temp_img[:,:,1], np.nanmin(temp_img[:,:,1]), np.nanmax(temp_img[:,:,1])) #normalize HV band
                    img= re_orderbands(temp_img[:,:,0], HV_norm, temp_img[:,:,1]) *255 #rearrange bands
                    img = img.astype(np.uint8) #convert values to int
                    img_ob = Image.fromarray(img) #convert to PIL Image object
                    img2 = preprocess(img_ob)
                    img2 = np.expand_dims(img2, 0) #Convert 2D image to 1D vector
                    img2 = torch.from_numpy(img2)

                    #classify image
                    inputs = Variable(img2).to(device)
                    inputs = inputs.float()
                    outputs = model(inputs) #labels
                    probabilities = torch.softmax(outputs.data, 1) #get probability for each class

                    if probabilities.cpu()[:,4] > 0.7: #classify as subsurface lake
                        img_ob.save(tiles_folder + f_name) #save image tile
                        num  = num + 1
                        lake_tile = temp_img[:,:,1] #get HV band
                        lake_tile = resize(lake_tile, (dx, dx),anti_aliasing=True) #resize   
                        th_img = thresholding(lake_tile) #thresholding
                        filtered_img = filter_lake(th_img) #filter lake
                        full_im[k:k+dx, j:j+dx] += filtered_img
                    elif probabilities.cpu()[:,6] > 0.75 or probabilities.cpu()[:,5] > 0.75: #there are mtns in image
                        mt.append([k,j,dx])


    del model
    cuda.empty_cache()
    gc.collect()
        
    full_im[full_im < 1] = 0
    full_im[full_im >= 1] = 1
    mtns = get_mtns(all_data, mt)

    #contour lakes
    contoured_lakes = measure.find_contours(full_im, 0.5) #contour 'lakes'
    if len(contoured_lakes) > 0:  
        print(f'There are potential lakes in this region')
        good_lakes, remaining_lakes = refine_lakes1(contoured_lakes)
        print(f'good lakes: {len(good_lakes)}, remaining: {len(remaining_lakes)}')
        good_lakes2 = refine_lakes2(remaining_lakes)
        print(f'new good lakes: {len(good_lakes2)}')
        lakes = lakes + good_lakes + good_lakes2
        print(f'final lakes: {len(lakes)}')
        print('done')
        del good_lakes
        del good_lakes2
        del remaining_lakes
    else: print(f'There are no lakes in this region')

    del full_im
    gc.collect()
    del mtns
    del all_data
  
   
intersection_lakes = list()
new_lakes = list()
for (i1, i2) in combinations(np.arange(len(lakes)), 2):  # 2 for pairs, 3 for triplets, etc
    l1 = Polygon(lakes[i1]['geometry']['coordinates'][0])
    l2 = Polygon(lakes[i2]['geometry']['coordinates'][0])
    if l1.intersects(l2):
        intersection_lakes.append(i1)
        intersection_lakes.append(i2)
        if l1.area < l2.area:
            new_lakes.append(lakes[i1])
        else:
            new_lakes.append(lakes[i2])

for i in range(len(lakes)):
    if i not in intersection_lakes:
        new_lakes.append(lakes[i])

print(len(new_lakes))
path = '/content/drive/My Drive/Lake detection/greenland/lakes/decrease/' + region + '_' + year + '.geojson'
list_to_json(new_lakes, path)
    

/content/drive/My Drive/Lake detection/greenland/2019/SE/original_data/SE10_2019_np.tif
loading model
loading data


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in less


256: classifying images
512: classifying images
There are potential lakes in this region
good lakes: 0, remaining: 10
new good lakes: 2
final lakes: 2
done
/content/drive/My Drive/Lake detection/greenland/2019/SE/original_data/SE11_2019_np.tif
loading model
loading data
256: classifying images
512: classifying images
There are potential lakes in this region


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:195: RuntimeWarning: Mean of empty slice


good lakes: 1, remaining: 13
new good lakes: 0
final lakes: 3
done
/content/drive/My Drive/Lake detection/greenland/2019/SE/original_data/SE12_2019_np.tif
loading model
loading data
256: classifying images
512: classifying images
There are potential lakes in this region
good lakes: 11, remaining: 18
new good lakes: 0
final lakes: 14
done
/content/drive/My Drive/Lake detection/greenland/2019/SE/original_data/SE13_2019_np.tif
loading model
loading data
256: classifying images
512: classifying images
There are potential lakes in this region
good lakes: 2, remaining: 4
new good lakes: 0
final lakes: 16
done
/content/drive/My Drive/Lake detection/greenland/2019/SE/original_data/SE14_2019_np.tif
loading model
loading data
256: classifying images
512: classifying images
There are potential lakes in this region
good lakes: 5, remaining: 6
new good lakes: 1
final lakes: 22
done
/content/drive/My Drive/Lake detection/greenland/2019/SE/original_data/SE15_2019_np.tif
loading model
loading data
256

In [ ]:
areas = 0
for lake in new_lakes:
    areas = areas + lake['properties']['area']
print(areas/(1000**2))

46.04304289858684
